In [3]:
import pandas as pd
import glob
data_path = 'data'

from azureml.core import Workspace
ws = Workspace.from_config()
ws.get_details()

{'id': '/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/issues/providers/Microsoft.MachineLearningServices/workspaces/issues',
 'name': 'issues',
 'location': 'northcentralus',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'sku': 'Basic',
 'workspaceid': '617d4e28-19eb-47e6-9710-14e2cfc8f1e4',
 'description': '',
 'friendlyName': '',
 'creationTime': '2020-01-10T13:33:11.1916827+00:00',
 'containerRegistry': '/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/issues/providers/Microsoft.ContainerRegistry/registries/issuesa24d4dd6',
 'keyVault': '/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourcegroups/issues/providers/microsoft.keyvault/vaults/issues3539678377',
 'applicationInsights': '/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourcegroups/issues/providers/microsoft.insights/components/issues5093552984',
 'identityPrincipalId': '75c4c007-89ad-4262-8739-998c0656c95b',
 'identityTenantId': '72f988bf-86f1-41

In [57]:
allFiles = glob.glob(data_path + "/issues-[0-9][0-9][0-9][0-9]-[0-9][0-9]-[0-9][0-9].csv")

list_ = []

for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    list_.append(df)

df = pd.concat(list_, axis = 0, ignore_index = True, sort=False)


In [58]:
#df = pd.read_csv(data_path + "/issues-latest.csv",index_col=None, header=0)

df['CreatedDate'] = pd.to_datetime(df['CreatedDate']) 
if df['CreatedDate'].dt.tz is None:
    df['CreatedDate'] = df['CreatedDate'].dt.tz_localize('UTC')

df['import_date'] = pd.to_datetime(df['import_date']).dt.tz_localize('UTC')
df['Age'] = (df['import_date'] - df['CreatedDate']).dt.days
df.loc[df['Age'] > 30, 'over30days'] = 1 
df.loc[df['Age'] <= 30, 'over30days'] = 0
df.loc[df['Age'] > 60, 'over60days'] = 1 
df.loc[df['Age'] <= 60, 'over60days'] = 0
df['over30days'] = df['over30days'].astype(int)
df['over60days'] = df['over60days'].astype(int)

df['TagsArray'] = df.Tags.str.split(';')
df['TagsArray'] = df.TagsArray.apply(lambda x: [y.strip().lower() for y in x])
df['papercut'] = df.TagsArray.apply(lambda x: 1 if 'papercut' in x or 'papercuts' in x else 0)
df['ws20ga'] = df.TagsArray.apply(lambda x: 1 if 'ws20ga' in x else 0)
df['customer'] = df.TagsArray.apply(lambda x: 1 if 'customer' in x else 0)

#test = df.query("target_month != target_month")[['target_month','triaged']]
prepped = df.fillna('nan')[['import_date', 'target_month', 'main_area', 'sub_area', 'triaged', 'State', 'Priority', 'over30days', 'over60days', 'papercut', 'ws20ga', 'customer', 'Age']]

grouped = prepped.groupby(['import_date', 'target_month', 'main_area', 'sub_area', 'triaged', 'State', 'Priority', 'over30days', 'over60days', 'papercut', 'ws20ga', 'customer'])
aggregated = grouped.size().reset_index(name='counts') 

age = grouped.sum().reset_index()['Age']
aggregated['SumAge'] = age

aggregated['hasMaxImportDate'] = (aggregated['import_date'] == max(aggregated['import_date']))


aggregated
#.target_month.unique()

,import_date,target_month,main_area,sub_area,triaged,State,Priority,over30days,over60days,papercut,ws20ga,customer,counts,SumAge,hasMaxImportDate
0,2020-01-11 00:00:00+00:00,nan,Assets,Environments,nan,New,2,0,0,1,0,0,1,2,False
1,2020-01-11 00:00:00+00:00,nan,Assets,Metastore,nan,New,2,1,0,1,0,0,1,45,False
2,2020-01-11 00:00:00+00:00,nan,Assets,Metastore,nan,New,4,0,0,1,0,0,1,29,False
3,2020-01-11 00:00:00+00:00,nan,Assets,Run History,nan,Done,2,0,0,1,0,0,1,22,False
4,2020-01-11 00:00:00+00:00,nan,Assets,Run History,nan,To Do,2,0,0,1,0,0,1,22,False
5,2020-01-11 00:00:00+00:00,nan,AutoML,AutoML,nan,Approved,2,0,0,1,0,0,1,17,False
6,2020-01-11 00:00:00+00:00,nan,AutoML,AutoML,nan,Approved,2,1,0,1,0,0,1,50,False
7,2020-01-11 00:00:00+00:00,nan,AutoML,AutoML,nan,Approved,2,1,1,1,0,0,1,98,False
8,2020-01-11 00:00:00+00:00,nan,AutoML,AutoML,nan,Approved,2,1,1,1,0,1,1,88,False
9,2020-01-11 00:00:00+00:00,nan,AutoML,AutoML,nan,Done,1,1,1,0,1,0,1,88,False


In [54]:
df = pd.read_csv(data_path + "/age-summary.csv",index_col=None, header=0)
df.query('triaged != triaged')


,import_date,target_month,main_area,sub_area,triaged,State,Priority,over30days,over60days,papercut,ws20ga,customer,counts,SumAge,hasMaxImportDate
0,2020-01-11 00:00:00+00:00,NaN,Assets,Environments,NaN,New,2,0,0,1,0,0,1,2,False
1,2020-01-11 00:00:00+00:00,NaN,Assets,Metastore,NaN,New,2,1,0,1,0,0,1,45,False
2,2020-01-11 00:00:00+00:00,NaN,Assets,Metastore,NaN,New,4,0,0,1,0,0,1,29,False
3,2020-01-11 00:00:00+00:00,NaN,Assets,Run History,NaN,Done,2,0,0,1,0,0,1,22,False
4,2020-01-11 00:00:00+00:00,NaN,Assets,Run History,NaN,To Do,2,0,0,1,0,0,1,22,False
5,2020-01-11 00:00:00+00:00,NaN,AutoML,AutoML,NaN,Approved,2,0,0,1,0,0,1,17,False
6,2020-01-11 00:00:00+00:00,NaN,AutoML,AutoML,NaN,Approved,2,1,0,1,0,0,1,50,False
7,2020-01-11 00:00:00+00:00,NaN,AutoML,AutoML,NaN,Approved,2,1,1,1,0,0,1,98,False
8,2020-01-11 00:00:00+00:00,NaN,AutoML,AutoML,NaN,Approved,2,1,1,1,0,1,1,88,False
9,2020-01-11 00:00:00+00:00,NaN,AutoML,AutoML,NaN,Done,1,1,1,0,1,0,1,88,False


In [71]:
if df['CreatedDate'].dt.tz is None:
    df['CreatedDate'] = df['CreatedDate'].dt.tz_localize('UTC')

In [72]:
start_time="2020-01-13T12:55:00"


False

In [82]:
outdf = pd.DataFrame(columns=df.columns)

for index, row in df.iterrows(): 
    row['foo'] = 'bar'
    outdf.append(row)
    
outdf

,AcceptanceCriteria,ActivatedBy,ActivatedDate,AreaPath,AssignedTo,BacklogPriority,BoardColumn,BoardColumnDone,BusinessValue,ChangedBy,...,id,import_date,main_area,relations,rev,sub_area,url,EpicStatus,StartDate,Status


In [74]:
pd.read_csv('data/age-summary.csv')

,import_date,id,main_area,sub_area,State,Priority,over30days,over60days,counts,SumAge,hasMaxImportDate
0,2020-01-10 00:00:00+00:00,410462,Core UI,Core UI,New,2,1,1,1,268,False
1,2020-01-10 00:00:00+00:00,433107,Core UI,Core UI,New,2,1,1,1,234,False
2,2020-01-10 00:00:00+00:00,437790,MLOps,MLOps,To Do,2,1,1,1,226,False
3,2020-01-10 00:00:00+00:00,472503,AutoML,SDK,Approved,2,1,1,1,178,False
4,2020-01-10 00:00:00+00:00,478101,AutoML,AutoML,In Progress,2,1,1,1,170,False
5,2020-01-10 00:00:00+00:00,478856,Training,Notebook Exp,New,2,1,1,1,169,False
6,2020-01-10 00:00:00+00:00,484656,MLOps,MLOps,To Do,2,1,1,1,160,False
7,2020-01-10 00:00:00+00:00,495765,MLOps,MLOps,Approved,2,1,1,1,141,False
8,2020-01-10 00:00:00+00:00,497142,Core UI,Notebooks,In Progress,2,1,1,1,139,False
9,2020-01-10 00:00:00+00:00,502493,Pipelines,AEva,To Do,2,1,1,1,128,False


In [14]:
df[['id','CreatedDate','Title','Tags','Description','AreaPath']]

,id,CreatedDate,Title,Tags,Description,AreaPath
0,580752,2020-01-08T19:28:15.413Z,No environment for data preparation,papercut,NaN,Vienna\Assets\Environments
1,571191,2019-12-19T22:17:54.497Z,"Run history: metrics are not showing up (AUC, ...",Papercuts; Run History; WS2.0,NaN,Vienna\Assets\Run History
2,571184,2019-12-19T22:10:46.81Z,Naming a run would be helpful to identify them...,Customer Engagement; Papercuts; Run History,<div>[Feedback from David Gollob]</div><div><b...,Vienna\Assets\Run History
3,478101,2019-07-23T21:57:43.217Z,Newer AutoML cannot load old saved model - bac...,ACE; Customer; papercut,"<b><u><a href=""#"" data-vss-mention=""version:2....",Vienna\AutoML
4,506920,2019-09-11T15:39:36.607Z,AutoML get_data() function,AVANADE; CHEVRON; Customer; papercut,<div>Took professional data scientists 4 days ...,Vienna\AutoML
5,582244,2020-01-10T00:11:33.15Z,[Automl tab] All run tables should have run nu...,papercut,NaN,Vienna\AutoML
6,582246,2020-01-10T00:12:00.897Z,[Automl tab] Runid columns should be abbreviated.,papercut,NaN,Vienna\AutoML
7,582247,2020-01-10T00:12:37.613Z,[Automl tab] The datetime format is bloated. W...,papercut,NaN,Vienna\AutoML
8,582250,2020-01-10T00:13:15.78Z,[Automl tab] The bread crumb is weird 'Welcome...,papercut,NaN,Vienna\AutoML
9,582251,2020-01-10T00:13:45.84Z,[Automl tab] 5 times we say 'Automated Machine...,papercut,NaN,Vienna\AutoML


In [4]:
from datetime import date, timedelta 
df['CreatedDate'] = pd.to_datetime(df['CreatedDate']).dt.tz_convert('UTC')
df['import_date'] = pd.to_datetime(df['import_date']).dt.tz_localize('UTC')
df['Age'] = (df['import_date'] - df['CreatedDate']).dt.days
df['over30days'] = df['Age'] > 30
df['over60days'] = df['Age'] > 60



In [6]:
gems = pd.read_csv(data_path + '/gems.csv')
df = pd.merge(df, gems, how='left', on=['AreaPath'])


In [7]:
df.shape

(18024, 46)

In [8]:
grouped = df.fillna('nan')[['import_date', 'GEM', 'EM', 'main_area', 'sub_area', 'State', 'Priority', 'over30days', 'over60days', 'Age']].groupby(['import_date', 'GEM', 'EM', 'main_area', 'sub_area', 'State', 'Priority', 'over30days', 'over60days'])
aggregated = grouped.size().reset_index(name='counts')
aggregated.head()

,import_date,GEM,EM,main_area,sub_area,State,Priority,over30days,over60days,counts
0,2019-03-02 00:00:00+00:00,Bert,Rajeev,UX,Model Management,Approved,2,True,True,5
1,2019-03-02 00:00:00+00:00,Bert,Rajeev,UX,Model Management,New,2,False,False,2
2,2019-03-02 00:00:00+00:00,Bert,Rajeev,UX,Model Management,New,2,True,False,1
3,2019-03-02 00:00:00+00:00,Bert,Rajeev,UX,Model Management,New,3,True,True,1
4,2019-03-02 00:00:00+00:00,Bert,Rajeev,UX,Reactbiza,Approved,2,True,True,6


In [9]:
sum(aggregated['counts'])

18024

In [10]:
age = grouped.sum().reset_index()['Age']
aggregated['SumAge'] = age

aggregated.head()

,import_date,GEM,EM,main_area,sub_area,State,Priority,over30days,over60days,counts,SumAge
0,2019-03-02 00:00:00+00:00,Bert,Rajeev,UX,Model Management,Approved,2,True,True,5,591
1,2019-03-02 00:00:00+00:00,Bert,Rajeev,UX,Model Management,New,2,False,False,2,27
2,2019-03-02 00:00:00+00:00,Bert,Rajeev,UX,Model Management,New,2,True,False,1,31
3,2019-03-02 00:00:00+00:00,Bert,Rajeev,UX,Model Management,New,3,True,True,1,73
4,2019-03-02 00:00:00+00:00,Bert,Rajeev,UX,Reactbiza,Approved,2,True,True,6,624


In [11]:
aggregated[['import_date','counts']].groupby(['import_date']).sum()

,counts
import_date,
2019-03-02 00:00:00+00:00,741
2019-03-03 00:00:00+00:00,744
2019-03-04 00:00:00+00:00,742
2019-03-05 00:00:00+00:00,737
2019-03-06 00:00:00+00:00,741
2019-03-07 00:00:00+00:00,748
2019-03-08 00:00:00+00:00,724
2019-03-09 00:00:00+00:00,732
2019-03-10 00:00:00+00:00,723


In [12]:
aggregated['hasMaxImportDate'] = aggregated['import_date'] == max(aggregated['import_date'])

In [13]:
aggregated

,import_date,GEM,EM,main_area,sub_area,State,Priority,over30days,over60days,counts,SumAge,hasMaxImportDate
0,2019-03-02 00:00:00+00:00,Bert,Rajeev,UX,Model Management,Approved,2,True,True,5,591,False
1,2019-03-02 00:00:00+00:00,Bert,Rajeev,UX,Model Management,New,2,False,False,2,27,False
2,2019-03-02 00:00:00+00:00,Bert,Rajeev,UX,Model Management,New,2,True,False,1,31,False
3,2019-03-02 00:00:00+00:00,Bert,Rajeev,UX,Model Management,New,3,True,True,1,73,False
4,2019-03-02 00:00:00+00:00,Bert,Rajeev,UX,Reactbiza,Approved,2,True,True,6,624,False
5,2019-03-02 00:00:00+00:00,Bert,Rajeev,UX,Reactbiza,New,2,True,True,4,390,False
6,2019-03-02 00:00:00+00:00,Bert,Salil,Model Management,Model Management,Approved,1,True,False,1,51,False
7,2019-03-02 00:00:00+00:00,Bert,Salil,Model Management,Model Management,Approved,1,True,True,1,95,False
8,2019-03-02 00:00:00+00:00,Bert,Salil,Model Management,Model Management,Approved,2,False,False,1,23,False
9,2019-03-02 00:00:00+00:00,Bert,Salil,Model Management,Model Management,Approved,2,True,False,2,101,False


In [ ]:
df.index

In [ ]:
aggregated.to_csv(data_path + '/bug-age.csv', index=False)

In [ ]:
reloaded = pd.read_csv(data_path + '/age-summary.csv')

In [ ]:
reloaded

In [23]:
gems = pd.DataFrame(dict(AreaPath = latest['AreaPath'].unique()))
gems['GEM'] = None
gems['EM'] = None

In [24]:
gems 

,AreaPath,GEM,EM
0,Vienna,None,None
1,Vienna\_archive,None,None
2,Vienna\1P Adoption,None,None
3,Vienna\AutoML,None,None
4,Vienna\Data Services\DataPrep,None,None
5,Vienna\Data Services\Datasets,None,None
6,Vienna\Data Services\Explainability,None,None
7,Vienna\DataDrift,None,None
8,Vienna\EPR,None,None
9,Vienna\Experimentation,None,None


In [94]:

elements = pd.read_csv('elements.csv')
elements

,AtomicNumber,Element,Symbol,AtomicMass,NumberofNeutrons,NumberofProtons,NumberofElectrons,Period,Group,Phase,...,FirstIonization,Density,MeltingPoint,BoilingPoint,NumberOfIsotopes,Discoverer,Year,SpecificHeat,NumberofShells,NumberofValence
0,1,Hydrogen,H,1.007,0,1,1,1,1.0,gas,...,13.5984,0.000090,14.175,20.28,3.0,Cavendish,1766.0,14.304,1,1.0
1,2,Helium,He,4.002,2,2,2,1,18.0,gas,...,24.5874,0.000179,NaN,4.22,5.0,Janssen,1868.0,5.193,1,NaN
2,3,Lithium,Li,6.941,4,3,3,2,1.0,solid,...,5.3917,0.534000,453.850,1615.00,5.0,Arfvedson,1817.0,3.582,2,1.0
3,4,Beryllium,Be,9.012,5,4,4,2,2.0,solid,...,9.3227,1.850000,1560.150,2742.00,6.0,Vaulquelin,1798.0,1.825,2,2.0
4,5,Boron,B,10.811,6,5,5,2,13.0,solid,...,8.2980,2.340000,2573.150,4200.00,6.0,Gay-Lussac,1808.0,1.026,2,3.0
5,6,Carbon,C,12.011,6,6,6,2,14.0,solid,...,11.2603,2.270000,3948.150,4300.00,7.0,Prehistoric,NaN,0.709,2,4.0
6,7,Nitrogen,N,14.007,7,7,7,2,15.0,gas,...,14.5341,0.001250,63.290,77.36,8.0,Rutherford,1772.0,1.040,2,5.0
7,8,Oxygen,O,15.999,8,8,8,2,16.0,gas,...,13.6181,0.001430,50.500,90.20,8.0,Priestley/Scheele,1774.0,0.918,2,6.0
8,9,Fluorine,F,18.998,10,9,9,2,17.0,gas,...,17.4228,0.001700,53.630,85.03,6.0,Moissan,1886.0,0.824,2,7.0
9,10,Neon,Ne,20.180,10,10,10,2,18.0,gas,...,21.5645,0.000900,24.703,27.07,8.0,Ramsay and Travers,1898.0,1.030,2,8.0


In [114]:

def year_for_element(element):
    number = elements.query(f"Element=='{element}'").AtomicNumber
    if len(number) != 1:
        return None
    else:
        return ((int(number)-1)//2)+2008


In [116]:
year_for_element("Manganese")

2020

In [109]:
((25-1)//2)+2008

2020